# Geração de Rosto
Neste projeto, você usará redes adversárias geradoras para gerar novas imagens de faces.
### Obtenha os dados
Você estará usando dois conjuntos de dados neste projeto:
- MNIST
- CelebA

Como o conjunto de dados celebA é complexo e você está fazendo GANs em um projeto pela primeira vez, queremos que você teste sua rede neural no MNIST antes da CelebA. Executar as GANs no MNIST permitirá que você veja o quão bem seu modelo treina mais cedo.

Se você estiver usando o [FloydHub] (https://www.floydhub.com/), defina `data_dir` como" / input "e use o [FloydHub data ID] (http://docs.floydhub.com/home / using_datasets /) "R5KrjnANiKVhLWAkpXhNBe".

In [27]:
data_dir = './data'

import helper

helper.download_extract('mnist', data_dir)
helper.download_extract('celeba', data_dir)

Found mnist Data


ConnectionResetError: [WinError 10054] Foi forçado o cancelamento de uma conexão existente pelo host remoto

## Explore os dados
### MNIST
Como você sabe, o conjunto de dados [MNIST] (http://yann.lecun.com/exdb/mnist/) contém imagens de dígitos manuscritos. Você pode ver o primeiro número de exemplos mudando `show_n_images`.

In [ ]:
show_n_images = 25

%matplotlib inline

import os
from glob import glob
from matplotlib import pyplot

mnist_images = helper.get_batch(glob(os.path.join(data_dir, 'mnist/*.jpg'))[:show_n_images], 28, 28, 'L')
pyplot.imshow(helper.images_square_grid(mnist_images, 'L'), cmap='gray')

### CelebA
O conjunto de dados [CelebA Attributes Dataset (CelebA)] (http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html) contém mais de 200.000 imagens de celebridades com anotações. Como você vai gerar faces, não precisará das anotações. Você pode ver o primeiro número de exemplos mudando `show_n_images`.

In [ ]:
show_n_images = 25

mnist_images = helper.get_batch(glob(os.path.join(data_dir, 'img_align_celeba/*.jpg'))[:show_n_images], 28, 28, 'RGB')
pyplot.imshow(helper.images_square_grid(mnist_images, 'RGB'))

## Pré-processar os dados
Como o foco principal do projeto é construir as GANs, nós pré-processaremos os dados para você. Os valores do conjunto de dados MNIST e CelebA estarão no intervalo de -0,5 a 0,5 de 28x28 imagens dimensionais. As imagens da CelebA serão cortadas para remover partes da imagem que não incluem um rosto e redimensionadas para 28x28.

As imagens MNIST são imagens em preto e branco com um único [canal de cor] (https://en.wikipedia.org/wiki/Channel_ (digital_image% 29), enquanto as imagens da CelebA têm [3 canais de cores (canal de cores RGB)] ( https://en.wikipedia.org/wiki/Channel_(digital_image%29#RGB_Images).
## Construa a Rede Neural
Você construirá os componentes necessários para construir uma GAN implementando as seguintes funções abaixo:
- `model_inputs`
- `discriminador`
- `gerador`
- `model_loss`
- `model_opt`
- `train`

### Verifique a versão do TensorFlow e acesse o GPU
Isso irá verificar se você tem a versão correta do TensorFlow e acesso a uma GPU

In [ ]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Verificar versão do Tensorflow
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Checar GPU
if not tf.test.gpu_device_name():
    warnings.warn('GPU Não encontrada. Por favor utilize uma GPU para treinar sua rede neural.')
else:
    print('GPU: {}'.format(tf.test.gpu_device_name()))

### Entrada
Implemente a função `model_inputs` para criar espaços reservados para TF para a rede neural. Deve criar os seguintes marcadores de posição:
- Espaçador de imagens de entrada real com classificação 4 usando `image_width`,` image_height` e `image_channels`.
- Espaço reservado de entrada Z com classificação 2 usando `z_dim`.
- Espaço reservado para a taxa de aprendizagem com classificação 0.

Retorna os marcadores de posição a seguir da tupla (tensor de imagens de entrada reais, tensor de dados z)

In [ ]:
import problem_unittests as tests

def model_inputs(image_width, image_height, image_channels, z_dim):
    input_feat = tf.placeholder(dtype=tf.float32, shape=(None, image_width, image_height, image_channels),
                              name='r_input')
    input_dim = tf.placeholder(dtype=tf.float32,shape=(None, z_dim), name='z_input')
    learning_rate = tf.placeholder(dtype=tf.float32, name='lr')

    return input_feat, input_dim, learning_rate

tests.test_model_inputs(model_inputs)

### Discriminador
Implemente `discriminator` para criar uma rede neural discriminadora que discrimine` images`. Esta função deve ser capaz de reutilizar as variáveis ​​na rede neural. Use [`tf.variable_scope`] (https://www.tensorflow.org/api_docs/python/tf/variable_scope) com um nome de escopo de" discriminador "para permitir que as variáveis ​​sejam reutilizadas. A função deve retornar uma tupla de (tensor output do discriminador, tensor logits do discriminador).

In [ ]:
def discriminator(images, reuse=False):
    
    with tf.variable_scope('discriminator', reuse=reuse):
        hl1 = tf.layers.conv2d(images, 64, 5, strides=2, padding='same')
        hl1 = tf.maximum(alpha * hl1, hl1)
        
        hl2 = tf.layers.conv2d(hl1, 128, 5, strides=2, padding='same')
        hl2 = tf.layers.batch_normalization(hl2, training=True)
        hl2 = tf.maximum(alpha * hl2, hl2)
        
        hl3 = tf.layers.conv2d(hl2, 256, 5, strides=2, padding='same')
        hl3 = tf.layers.batch_normalization(hl3, training=True)
        hl3 = tf.maximum(alpha * hl3, hl3)

        inp_f = tf.reshape(h3, (-1, 4*4*256))
        dl = tf.layers.dropout(inputs=inp_f, rate=dropout)
        logits = tf.layers.dense(dl, 1)
        output = tf.sigmoid(logits)
        
    return output, logits

tests.test_discriminator(discriminator, tf)

### Gerador
Implemente `generator` para gerar uma imagem usando` z`. Esta função deve ser capaz de reutilizar as variáveis ​​na rede neural. Use [`tf.variable_scope`] (https://www.tensorflow.org/api_docs/python/tf/variable_scope) com um nome de escopo de" generator "para permitir que as variáveis ​​sejam reutilizadas. A função deve retornar as imagens 28 x 28 x `out_channel_dim` geradas.

In [ ]:
def generator(z, out_channel_dim, is_train=True):
    with tf.variable_scope('generator', reuse= not is_train):
        hl1 = tf.layers.dense(z, 3*3*512) # Dimensões
        hl1 = tf.reshape(h1, (-1, 3, 3, 512)) # Novo shape
        hl1 = tf.layers.batch_normalization(hl1, training=is_train)
        hl1 = tf.maximum(alpha * hl1, hl1)
        
        hl2 = tf.layers.conv2d_transpose(hl1, 256, kernel_size=4, strides=2, padding='same')
        hl2 = tf.layers.batch_normalization(hl2, training=is_train)
        hl2 = tf.maximum(alpha * hl2, hl2)
        
        hl3 = tf.layers.conv2d_transpose(hl2, 128, kernel_size=4, strides=2, padding='valid')
        hl3 = tf.layers.batch_normalization(hl3, training=is_train)
        hl3 = tf.maximum(alpha * hl3, hl3)
        
        logits = tf.layers.conv2d_transpose(hl3, out_channel_dim, kernel_size=5,strides=2, padding='same')
        output = tf.tanh(logits)
        
        return output

tests.test_generator(generator, tf)

### Perda
Implemente `model_loss` para construir as GANs para treinamento e calcular a perda. A função deve retornar uma tupla de (perda do discriminador, perda do gerador). Use as seguintes funções que você implementou:
- `discriminador (imagens, reutilização = Falso)`
- `gerador (z, out_channel_dim, is_train = True)`

In [ ]:
def model_loss(input_real, input_z, out_channel_dim):

    generator = generator(input_z, out_channel_dim, is_train=True, alpha=alpha)
    discriminator, r_logits = discriminator(input_real, reuse=False, alpha=alpha, dropout=dropout)
    f_model, f_logits = discriminator(generator, reuse=True, alpha=alpha, dropout=dropout)

    r_loss = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_r, 
                                                labels=tf.ones_like(discriminator)*(1 - smooth)))
    f_loss = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=f_logits, labels=tf.zeros_like(f_model)))
    g_loss = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=f_logits, labels=tf.ones_like(f_model)))

    d_loss = r_loss + f_loss

    return d_loss, g_loss
    return None, None

tests.test_model_loss(model_loss)

### Otimização
Implemente `model_opt` para criar as operações de otimização para as GANs. Use [`tf.trainable_variables`] (https://www.tensorflow.org/api_docs/python/tf/trainable_variables) para obter todas as variáveis ​​treináveis. Filtre as variáveis ​​com nomes que estão nos nomes do escopo do discriminador e do gerador. A função deve retornar uma tupla de (operação de treinamento do discriminador, operação de treinamento do gerador).

In [ ]:
def model_opt(d_loss, g_loss, learning_rate, beta1):
    train_var = tf.trainable_variables()
    discriminator_var = [var for var in train_var if var.name.startswith('discriminator')]
    generator_var = [var for var in train_var if var.name.startswith('generator')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
        dis_train_opt = tf.train.AdamOptimizer(learning_rate, beta1=beta1).minimize(d_loss, var_list=discriminator_var)
        gen_train_opt = tf.train.AdamOptimizer(learning_rate, beta1=beta1).minimize(g_loss, var_list=generator_var)

    return d_train_opt, g_train_opt

tests.test_model_opt(model_opt, tf)

## Treinamento em Redes Neurais
### Show Output
Use esta função para mostrar a saída atual do gerador durante o treinamento. Isso ajudará você a determinar quão bem as GANs estão treinando.

In [ ]:
import numpy as np

def show_generator_output(sess, n_images, input_z, out_channel_dim, image_mode):
    cmap = None if image_mode == 'RGB' else 'gray'
    z_dim = input_z.get_shape().as_list()[-1]
    example_z = np.random.uniform(-1, 1, size=[n_images, z_dim])

    samples = sess.run(
        generator(input_z, out_channel_dim, False),
        feed_dict={input_z: example_z})

    images_grid = helper.images_square_grid(samples, image_mode)
    pyplot.imshow(images_grid, cmap=cmap)
    pyplot.show()

### Treinar
Implementar o `trem` para construir e treinar as GANs. Use as seguintes funções que você implementou:
- `model_inputs (image_width, image_height, image_channels, z_dim)`
- `model_loss (input_real, input_z, out_channel_dim)`
- `model_opt (d_loss, g_loss, learning_rate, beta1)`

Use o `show_generator_output` para mostrar a saída do` generator` enquanto você treina. Executar `show_generator_output` para cada lote aumentará drasticamente o tempo de treinamento e aumentará o tamanho do bloco de anotações. É recomendado imprimir a saída do `gerador` a cada 100 lotes.

In [ ]:
def train(epoch_count, batch_size, z_dim, learning_rate, beta1, get_batches, data_shape, data_image_mode):

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch_i in range(epoch_count):
            for batch_images in get_batches(batch_size):
                # TODO: Train Model
                
                

### MNIST
Teste sua arquitetura de GANs no MNIST. Depois de 2 épocas, as GANs devem poder gerar imagens que se pareçam com dígitos manuscritos. Certifique-se de que a perda do gerador seja menor que a perda do discriminador ou próxima de 0.

In [ ]:
batch_size = None
z_dim = None
learning_rate = None
beta1 = None

epochs = 2

mnist_dataset = helper.Dataset('mnist', glob(os.path.join(data_dir, 'mnist/*.jpg')))
with tf.Graph().as_default():
    train(epochs, batch_size, z_dim, learning_rate, beta1, mnist_dataset.get_batches,
          mnist_dataset.shape, mnist_dataset.image_mode)

### CelebA
Execute suas GANs na CelebA. Demora cerca de 20 minutos na GPU média para executar uma época. Você pode executar toda a época ou parar quando começar a gerar faces realistas.

In [ ]:
batch_size = None
z_dim = None
learning_rate = None
beta1 = None

epochs = 1

celeba_dataset = helper.Dataset('celeba', glob(os.path.join(data_dir, 'img_align_celeba/*.jpg')))
with tf.Graph().as_default():
    train(epochs, batch_size, z_dim, learning_rate, beta1, celeba_dataset.get_batches,
          celeba_dataset.shape, celeba_dataset.image_mode)

### Enviando este projeto
Ao enviar este projeto, certifique-se de executar todas as células antes de salvar o bloco de anotações. Salve o arquivo do notebook como "dlnd_face_generation.ipynb" e salve-o como um arquivo HTML em "File" -> "Download as". Inclua os arquivos "helper.py" e "problem_unittests.py" no seu envio.